In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

# 获取郑州商品交易所（郑商所）的历史数据

[郑商所官网](http://www.czce.com.cn/)顶部菜单“交易数据” -> “历史行情下载”。

鼠标在下图红框示意的位置上划过，就能在浏览器的状态栏看到下载地址。

![郑商所历史数据](../pictures/czce_data.png)

看起来不如上期所和中金所的下载链接有规律。

但是，只要是浏览器可以获得的公开数据（比如没有密码什么的），我们的爬虫一样可以获得。区别就是解析数据的繁琐程度。

郑商所的下载链接需要解析网页了。

安装 lxml 第三方库：

`pip install lxml`

解析网页可以一次性的获取期货历史数据下载链接和期权历史数据下载链接，需要做个区分。

代码如下：

In [2]:
from typing import Dict

import requests
from lxml import etree

def fetch_czce_history_index() -> Dict[str, Dict[int, str]]:
    """
    获取郑州商品交易所（郑商所，CZCE）的历史数据下载链接。
    :return: Dict[str, Dict[int, str]]，一个嵌套的字典。
             外层字典的 key 是期货（'futures'）或者期权（'option'），
             内层字典的 key 是年份，value 是下载地址。
    """
    
    # 定义将返回的结果的数据结构。
    result: Dict[str, Dict[int, str]] = {
        'futures': {},
        'option': {},
    }
    
    # 郑商所 url 和历史行情 url。
    url_czce: str = 'http://www.czce.com.cn'
    url: str = f'{url_czce}/cn/jysj/lshqxz/H770319index_1.htm'

    # 下载历史行情页。
    response = requests.get(url)
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(
            f'下载 <{url}> 时发生错误。'
        )
    response.encoding = 'utf-8'

    # 解析网页。
    html: etree._Element = etree.HTML(response.text)
    url_list: List[str] = []
    year_list: List[str] = []
    node_list: List[etree._Element] = html.xpath('//li/span[@class="hidden-xs"]')
    for node in node_list:
        url_list.append(node.xpath('./a[@target="_blank"]/@href')[0])
        year_list.append(node.xpath('../text()')[0])
    assert len(url_list) == len(year_list)

    # 填充数据.
    for i in range(len(url_list)):
        year = int(year_list[i][:4])
        if 'Option' in url_list[i]:
            result['option'][year] = f'{url_czce}{url_list[i]}'
        else:
            result['futures'][year] = f'{url_czce}{url_list[i]}'
    return result

# 测试。
print(fetch_czce_history_index())

{'futures': {2021: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2021/ALLFUTURES2021.zip', 2020: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2020/ALLFUTURES2020.zip', 2019: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2019/FutureDataHistory.zip', 2018: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2018/FutureDataHistory.zip', 2017: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2017/FutureDataHistory.zip', 2016: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2016/FutureDataHistory.zip', 2015: 'http://www.czce.com.cn/cn/DFSStaticFiles/Future/2015/FutureDataHistory.zip', 2014: 'http://www.czce.com.cn/cn/exchange/datahistory2014.zip', 2013: 'http://www.czce.com.cn/cn/exchange/datahistory2013.zip', 2012: 'http://www.czce.com.cn/cn/exchange/datahistory2012.zip', 2011: 'http://www.czce.com.cn/cn/exchange/datahistory2011.zip', 2010: 'http://www.czce.com.cn/cn/exchange/datahistory2010.zip'}, 'option': {2021: 'http://www.czce.com.cn/cn/DFSStaticFiles/Option/2021/ALLOPTIONS202

下载历史数据的代码：

In [3]:
def download_czce_history_data(save_path: Path, year: int, is_futures: bool = True):
    """
    下载郑州商品交易所（郑商所，CZCE）的历史数据。
    :param save_path: Path，保存的位置。
    :param year: int，需要下载数据的年份。
    :param is_futures: bool，为 True 表示期货，否则表示期权。默认是期货。
    :return: None。
    """
    # 获取下载链接。
    url_mapper: Dict[str, Dict[int, str]] = fetch_czce_history_index()
    
    # 获得本次下载的内容（期货或者期权）。
    type_: str
    if is_futures:
        type_ = 'futures'
    else:
        type_ = 'option'
    url_list: Dict[int, str] = url_mapper[type_]

    # 如果参数 <year> 不在本次下载内容中，引发异常。
    if year not in url_list.keys():
        raise ValueError('<year> is beyond possible range.')

    # 如果参数 <save_path> 不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')

    # 下载
    response = requests.get(url_list[year])
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(f'下载 <{url_list[year].format(year=year)}> 时发生错误。')
    
    # 保存文件。
    with open(save_path.joinpath(f'CZCE_{type_}_{year:4d}.zip'), 'wb') as f:
        f.write(response.content)

# 测试。
download_czce_history_data(save_path=PATH_FOR_RESULT, year=2020, is_futures=True)

下载所有历史数据的代码：

In [4]:
def download_all_czce_history_data(save_path: Path) -> None:
    """
    下载郑州商品交易所（郑商所，CZCE）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    
    # 如果参数 <save_path> 不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')
    
    # 获取下载链接。
    url_mapper: Dict[str, Dict[int, str]] = fetch_czce_history_index()
    
    # 循环，分别下载期货和期权。
    for type_ in ['futures', 'option']:
        
        # 按年份循环。
        for year in url_mapper[type_].keys():
            
            # 具体的 url。
            url = url_mapper[type_][year]
            
            # 下载
            response = requests.get(url)
            
            # 如果下载不顺利，引发异常。
            if response.status_code != 200:
                raise requests.exceptions.HTTPError(f'下载 <{url}> 时发生错误。')
            
            # 保存文件。
            with open(save_path.joinpath(f'CZCE_{type_}_{year:4d}.zip'), 'wb') as f:
                f.write(response.content)

# 测试。
download_all_czce_history_data(save_path=PATH_FOR_RESULT)

Done.